In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogus-datathon-otomotiv/MASK_SERVIS_BAKIM_DATATHON_FINAL.csv
/kaggle/input/dogus-datathon-otomotiv/sample_submission.csv
/kaggle/input/dogus-datathon-otomotiv/FINAL_SIFIR_ARAC_ALANLAR_DATATHON.csv
/kaggle/input/dogus-datathon-otomotiv/FINAL_SALES_FILE_DATATHON.csv
/kaggle/input/dogus-datathon-otomotiv/FINAL_VEHICLE_TABLE_DATATHON.csv
/kaggle/input/dogus-datathon-otomotiv/FINAL_CUSTOMER_DATATHON.csv
/kaggle/input/dogus-datathon-otomotiv/FINAL_CUSTOMER_RELATED_TABLE_FOR_DATATHON.csv


In [2]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

In [3]:
base = '/kaggle/input/dogus-datathon-otomotiv/'
main_df_raw = pd.read_csv(os.path.join(base,'FINAL_CUSTOMER_DATATHON.csv' ))
main_df = main_df_raw.drop(columns=['Unnamed: 0', 'GENDER_ID', 'MARITAL_STATUS_ID',])
#Cinsiyet bilgisini yeniden kodlama: 
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849919 entries, 0 to 849918
Data columns (total 7 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   BASE_CUSTOMER_ID               849919 non-null  int64  
 1   CUSTOMER_ID                    849919 non-null  int64  
 2   GENDER                         849916 non-null  object 
 3   MARITAL_STATUS                 620802 non-null  object 
 4   BIRTH_DATE                     645447 non-null  float64
 5   FK_ADDRESS_COMMUNICATION_CITY  730341 non-null  object 
 6   OCCUPATION                     381052 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 45.4+ MB


In [4]:
sales_df_bare = pd.read_csv(os.path.join(base, 'FINAL_SALES_FILE_DATATHON.csv'))
sales_df = sales_df_bare.drop(columns=['Unnamed: 0'])
buying_customers = sales_df['CUSTOMER_ID'].unique() #Sadece satış sayısın bakrarak
buying_customers

array([1712792, 1149729, 1697993, ..., 1383901, 1498539, 1649602])

In [5]:
def has_sales(cid, buying_customers):
    if(cid in buying_customers):
        return 1
    else:
        return 0

main_df['VAL'] = main_df.apply (lambda row: has_sales(row.CUSTOMER_ID, buying_customers), axis=1)

In [6]:
categorical = ['GENDER','MARITAL_STATUS','FK_ADDRESS_COMMUNICATION_CITY','OCCUPATION']
data_2 = main_df[['BASE_CUSTOMER_ID','CUSTOMER_ID', 'BIRTH_DATE']].copy()

# iterative imputation transform for the horse colic dataset

# evaluate iterative imputation and random forest for the horse colic dataset



from numpy import isnan
from pandas import read_csv
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# split into input and output elements

data = data_2.values
ix = [i for i in range(data.shape[1]) if i != 3]
X, y = data[:, ix], data[:, 2]
# print total missing
print('Missing: %d' % sum(isnan(X).flatten()))
# define imputer
imputer = IterativeImputer()
# fit on the dataset
imputer.fit(X)
# transform the dataset
Xtrans = imputer.transform(X)
# print total missing
print('Missing: %d' % sum(isnan(Xtrans).flatten()))
Xtrans

Missing: 204472
Missing: 0


array([[ 158891.        , 1747700.        ,    1972.80351245],
       [ 169233.        , 1710561.        ,    1962.        ],
       [  30887.        , 1043923.        ,    1971.        ],
       ...,
       [ 155926.        , 1670812.        ,    1970.        ],
       [ 123223.        , 1691689.        ,    1970.        ],
       [  88245.        , 1300543.        ,    1966.        ]])

In [7]:
for a in categorical:
    if a == categorical[0]:
        mylist_1 = list(set(main_df[a]))
    elif a == categorical[1]:
        mylist_2 = list(set(main_df[a]))
    elif a == categorical[2]:    
        mylist_3 = list(set(main_df[a]))
    else:
        mylist_4 = list(set(main_df[a]))

mylist = list(set(main_df["OCCUPATION"]))
newlist_1 = [x for x in mylist_1 if pd.isnull(x) == False and x != 'nan']
newlist_2 = [x for x in mylist_2 if pd.isnull(x) == False and x != 'nan']
newlist_3 = [x for x in mylist_3 if pd.isnull(x) == False and x != 'nan']
newlist_4 = [x for x in mylist_4 if pd.isnull(x) == False and x != 'nan']

In [8]:
main_df["GENDER"] = main_df["GENDER"].fillna(pd.Series(np.random.choice(newlist_1, size=len(main_df.index))))
main_df["MARITAL_STATUS"] = main_df["MARITAL_STATUS"].fillna(pd.Series(np.random.choice(newlist_2, size=len(main_df.index))))
main_df["FK_ADDRESS_COMMUNICATION_CITY"] = main_df["FK_ADDRESS_COMMUNICATION_CITY"].fillna(pd.Series(np.random.choice(newlist_3, size=len(main_df.index))))
main_df["OCCUPATION"] = main_df["OCCUPATION"].fillna(pd.Series(np.random.choice(newlist_4, size=len(main_df.index))))
# used to check if all nans are now gone.
"""
a = 0
for i in main_df["OCCUPATION"]:
    if i == 'nan':
        a += 1
        
print(a)
"""

'\na = 0\nfor i in main_df["OCCUPATION"]:\n    if i == \'nan\':\n        a += 1\n        \nprint(a)\n'

In [9]:
main_df["BASE_CUSTOMER_ID"] = Xtrans[:,0]
# checked the values main_df["BASE_CUSTOMER_ID"]
main_df["CUSTOMER_ID"] = Xtrans[:,1]
main_df["BIRTH_DATE"] = Xtrans[:,2]

In [10]:
main_df["BASE_CUSTOMER_ID"] = main_df["BASE_CUSTOMER_ID"].astype(np.int64)
main_df["CUSTOMER_ID"] = main_df["CUSTOMER_ID"].astype(np.int64)
main_df["BIRTH_DATE"] = main_df["BIRTH_DATE"].astype(np.int64)

In [11]:
df = main_df.copy()
categorical = ['GENDER','MARITAL_STATUS','FK_ADDRESS_COMMUNICATION_CITY','OCCUPATION']

from sklearn import preprocessing
for col in categorical:
    le = preprocessing.LabelEncoder()
    column = df[col].copy()
    df[col] = le.fit_transform(column)

df

,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION,VAL
0,158891,1747700,0,0,1972,43,48,1
1,169233,1710561,0,1,1962,41,57,1
2,30887,1043923,0,1,1971,137,14,0
3,38013,1101926,0,0,1963,81,58,0
4,157091,1819787,1,1,1973,8,19,0
...,...,...,...,...,...,...,...,...
849914,18175,1744051,0,1,1965,114,57,0
849915,108604,1209486,0,1,1969,137,34,1
849916,155926,1670812,0,1,1970,137,19,0
849917,123223,1691689,0,1,1970,64,37,0


In [12]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
y_train = df['VAL'].to_numpy() 
X_train = df.drop(columns = ['VAL', 'BASE_CUSTOMER_ID', 'CUSTOMER_ID']).to_numpy()
X_train

array([[   0,    0, 1972,   43,   48],
       [   0,    1, 1962,   41,   57],
       [   0,    1, 1971,  137,   14],
       ...,
       [   0,    1, 1970,  137,   19],
       [   0,    1, 1970,   64,   37],
       [   1,    0, 1966,  137,   49]])

In [13]:
id_df = pd.read_csv(os.path.join(base, 'sample_submission.csv'))[['Id']]

#Only predicting for people with full values
X = df[df.BASE_CUSTOMER_ID.isin(id_df['Id'].to_numpy())].copy()
X_test = X.copy().drop(columns = ['VAL', 'BASE_CUSTOMER_ID', 'CUSTOMER_ID']).to_numpy()

print("Total result size: " + str(id_df.shape[0]))

print("Predicted result size: " + str(X.shape[0]))

print("df size: " + str(df.shape[0]))

Total result size: 166943
Predicted result size: 714249
df size: 849919


In [14]:
logmodel.fit(X_train,y_train)
predictions = logmodel.predict_proba(X_test)
p_vals = []

for pred in predictions:
    p_vals.append(pred[1])

X['P'] = p_vals #TODO: Fix warning here


#TODO: Fix classificaiton imbalance before calling the scikit learn! It causes us to clssify everything as 0.

In [15]:
X2 = X[['BASE_CUSTOMER_ID','P']].copy().rename(columns={"BASE_CUSTOMER_ID": "Id", "P": "Expected"})
X2

result = id_df.merge(X2, on='Id', how='left')
result['Expected'] = result['Expected'].fillna((result['Expected'].mean()))
result.to_csv('/kaggle/working/predictions.csv',index=False)
